<a href="https://colab.research.google.com/github/andrepps22/Data_Science/blob/main/Anlise_valor_fechamento_MXRF11_D%2B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lendo o arquivo do MXRF1


In [ ]:
import pandas as pd

In [54]:
df = pd.read_csv('/content/MXRF11 Historical Data.csv')

In [67]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,02/14/2024,10.44,10.52,10.52,10.41,1.35M,-0.85%
1,02/09/2024,10.53,10.49,10.53,10.48,1.15M,0.38%
2,02/08/2024,10.49,10.51,10.52,10.49,1.03M,0.00%
3,02/07/2024,10.49,10.49,10.53,10.48,1.25M,0.10%
4,02/06/2024,10.48,10.42,10.50,10.42,1.39M,0.58%


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2501 non-null   object 
 1   Price     2501 non-null   float64
 2   Open      2501 non-null   float64
 3   High      2501 non-null   float64
 4   Low       2501 non-null   float64
 5   Vol.      2498 non-null   object 
 6   Change %  2501 non-null   object 
dtypes: float64(4), object(3)
memory usage: 136.9+ KB


Como são somentes três dados que estão nulos na colula "Vol." vamos completar com os valores que mais aparece

In [59]:
df[df['Vol.'].isnull()]

,Date,Price,Open,High,Low,Vol.,Change %
105,09/09/2023,10.87,10.87,10.87,10.87,NaN,0.09%
110,09/02/2023,10.79,10.79,10.79,10.79,NaN,0.28%
195,05/06/2023,10.52,10.52,10.52,10.52,NaN,0.00%


In [60]:
df['Vol.'].value_counts()

1.06M      6
1.04M      6
1.10M      5
1.11M      5
1.31M      5
          ..
658.64K    1
399.34K    1
619.07K    1
580.48K    1
91.72K     1
Name: Vol., Length: 2353, dtype: int64

In [61]:
df['Vol.'].fillna('1.06M', inplace=True)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2501 non-null   object 
 1   Price     2501 non-null   float64
 2   Open      2501 non-null   float64
 3   High      2501 non-null   float64
 4   Low       2501 non-null   float64
 5   Vol.      2501 non-null   object 
 6   Change %  2501 non-null   object 
dtypes: float64(4), object(3)
memory usage: 136.9+ KB


Separando os dados que queremos prever

queremos prever se o preço vai ser maior que a abertura no final do dia seqguinte

In [64]:
mx = df[['Open', 'High', 'Low', 'Price']]

In [ ]:
mx['Vol.'] = df['Vol.'].str.replace('M', '', regex=True).str.replace('K', '', regex=True).astype(float)
mx['Change %'] = df['Change %'].replace('%', '',regex=True).astype(float) / 100

In [66]:
mx

,Open,High,Low,Price,Vol.,Change %
0,10.52,10.52,10.41,10.44,1.35,-0.0085
1,10.49,10.53,10.48,10.53,1.15,0.0038
2,10.51,10.52,10.49,10.49,1.03,0.0000
3,10.49,10.53,10.48,10.49,1.25,0.0010
4,10.42,10.50,10.42,10.48,1.39,0.0058
...,...,...,...,...,...,...
2496,3.12,3.16,3.05,3.12,84.04,0.0000
2497,3.22,3.22,3.12,3.12,137.65,-0.0311
2498,3.26,3.26,3.20,3.22,121.85,-0.0123
2499,3.32,3.32,3.25,3.26,97.95,-0.0181


In [68]:
mx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      2501 non-null   float64
 1   High      2501 non-null   float64
 2   Low       2501 non-null   float64
 3   Price     2501 non-null   float64
 4   Vol.      2501 non-null   float64
 5   Change %  2501 non-null   float64
dtypes: float64(6)
memory usage: 117.4 KB


In [73]:
#Criando a variavel targe e retirando o ultimo valor da variavel preço, a ultima linha que vai ficar com o do nulo será removida
#isso é necessario para que consigamos prever o valor no dia seguinte do ativo
mx['Target'] = mx.loc[1:mx.shape[0], 'Open'].reset_index(drop=True)

In [74]:
mx.tail()

,Open,High,Low,Price,Vol.,Change %,Target
2496,3.12,3.16,3.05,3.12,84.04,0.0000,3.22
2497,3.22,3.22,3.12,3.12,137.65,-0.0311,3.26
2498,3.26,3.26,3.20,3.22,121.85,-0.0123,3.32
2499,3.32,3.32,3.25,3.26,97.95,-0.0181,3.37
2500,3.37,3.37,3.28,3.32,91.72,-0.0090,NaN


In [75]:
amanha = mx.loc[2500]
amanha

Open         3.370
High         3.370
Low          3.280
Price        3.320
Vol.        91.720
Change %    -0.009
Target         NaN
Name: 2500, dtype: float64

In [76]:
mx.drop(2500)

,Open,High,Low,Price,Vol.,Change %,Target
0,10.52,10.52,10.41,10.44,1.35,-0.0085,10.49
1,10.49,10.53,10.48,10.53,1.15,0.0038,10.51
2,10.51,10.52,10.49,10.49,1.03,0.0000,10.49
3,10.49,10.53,10.48,10.49,1.25,0.0010,10.42
4,10.42,10.50,10.42,10.48,1.39,0.0058,10.54
...,...,...,...,...,...,...,...
2495,3.12,3.12,3.08,3.08,117.18,-0.0128,3.12
2496,3.12,3.16,3.05,3.12,84.04,0.0000,3.22
2497,3.22,3.22,3.12,3.12,137.65,-0.0311,3.26
2498,3.26,3.26,3.20,3.22,121.85,-0.0123,3.32


Realizando a transformação da variavel targer se o valor do ativo for maior que o da abertura vai receber o numero 1 caso seja menor ou igual vai receber o 0

In [77]:
mx.tail()

,Open,High,Low,Price,Vol.,Change %,Target
2496,3.12,3.16,3.05,3.12,84.04,0.0000,3.22
2497,3.22,3.22,3.12,3.12,137.65,-0.0311,3.26
2498,3.26,3.26,3.20,3.22,121.85,-0.0123,3.32
2499,3.32,3.32,3.25,3.26,97.95,-0.0181,3.37
2500,3.37,3.37,3.28,3.32,91.72,-0.0090,NaN


In [78]:
mx.loc[mx['Open'] > mx['Target'], 'Target'] = 1

In [79]:
mx.loc[mx['Target'] != 1, 'Target'] = 0

In [81]:
mx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      2501 non-null   float64
 1   High      2501 non-null   float64
 2   Low       2501 non-null   float64
 3   Price     2501 non-null   float64
 4   Vol.      2501 non-null   float64
 5   Change %  2501 non-null   float64
 6   Target    2501 non-null   float64
dtypes: float64(7)
memory usage: 136.9 KB


In [82]:
y = mx['Target']
X = mx.drop('Target', axis = 1)

In [83]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2501 entries, 0 to 2500
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      2501 non-null   float64
 1   High      2501 non-null   float64
 2   Low       2501 non-null   float64
 3   Price     2501 non-null   float64
 4   Vol.      2501 non-null   float64
 5   Change %  2501 non-null   float64
dtypes: float64(6)
memory usage: 117.4 KB


#Treinamento do algoritimo e normalização

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [85]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=123)


In [86]:
X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape

((1750, 6), (751, 6), (1750,), (751,))

In [91]:
std = StandardScaler()
X_std_treino = std.fit_transform(X_treino)
X_std_teste = std.fit_transform(X_teste)


##ExtraTreesClassifier

In [101]:
from sklearn.ensemble import ExtraTreesClassifier

In [102]:
modelo = ExtraTreesClassifier()
modelo.fit(X_std_treino, y_treino)

ExtraTreesClassifier()

In [103]:
resultado = modelo.score(X_std_teste, y_teste)

In [104]:
resultado

0.5193075898801598

##RandomForestClassifier

In [105]:
from sklearn.ensemble import RandomForestClassifier

In [106]:
rfc = RandomForestClassifier()
rfc.fit(X_std_treino, y_treino)

RandomForestClassifier()

In [107]:
rfc.score(X_std_teste, y_teste)

0.5193075898801598

##SVM

In [108]:
from sklearn.svm import SVC

In [110]:
svm = SVC()
svm.fit(X_std_treino, y_treino)

SVC()

In [111]:
svm.score(X_std_teste, y_teste)

0.559254327563249